# Chapter 6

# Section 6.3.2 Subsample Analysis

# VAR-BEKK-GARCH model for Period 2 (2011/10 - 2015/11) (Section 6.3.2)

In [1]:
library(xts)
library(BEKKs)
library(MTS)
library(vars)
library(stargazer)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘MTS’


The following object is masked from ‘package:BEKKs’:

    backtest


Loading required package: MASS

Loading required package: strucchange

Loading required package: sandwich

Loading required package: urca

Loading required package: lmtest


Attaching package: ‘vars’


The following object is masked from ‘package:MTS’:

    VAR



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




# Dataset Preperation

In [2]:
fr_d <- readRDS("fr_d.Rds")
fp_d <- readRDS("fp_d.Rds")
cr_d <- readRDS("cr_d.Rds")
cp_d <- readRDS("cp_d.Rds")
head(cr_d)
head(fr_d)
us_d <- cr_d[,1]
l_d <- cr_d[,2]

                     usc           lc
2007-07-06  0.0033026687  0.005272420
2007-07-09 -0.0108928380 -0.013233541
2007-07-10 -0.0378502241 -0.040784991
2007-07-11  0.0210431058  0.019240172
2007-07-12  0.0057943185  0.004526943
2007-07-13 -0.0009633912 -0.004526943

                  eurusd        gbpusd        chfusd        usdgbp       eurgbp
2007-07-06  2.240547e-03 -0.0004972527 -0.0003044418  0.0007037652  0.002659380
2007-07-09  7.337565e-05  0.0022605055  0.0012172115 -0.0025157246 -0.001993871
2007-07-10  8.584371e-03  0.0057647029  0.0115503355 -0.0056582954  0.002805051
2007-07-11 -7.274844e-05  0.0027838359 -0.0029504763 -0.0026379885 -0.002952903
2007-07-12  3.485969e-03 -0.0009845428  0.0028903495  0.0008124302  0.004573291
2007-07-13 -7.252421e-04  0.0019435396  0.0000000000 -0.0020323145 -0.003391083
                  chfgbp        ghsusd        ghsgbp
2007-07-06  0.0001273483 -0.0033264938 -2.614079e-03
2007-07-09 -0.0009873737  0.0040086898  1.236683e-03
2007-07-10  0.0059364761 -0.0002705906 -5.801912e-03
2007-07-11 -0.0057330414  0.0026887042  4.994937e-05
2007-07-12  0.0037670817  0.0027212691  3.716576e-03
2007-07-13 -0.0020261005 -0.0051426368 -7.210559e-03

In [3]:
head(fr_d)
head(fp_d)

                  eurusd        gbpusd        chfusd        usdgbp       eurgbp
2007-07-06  2.240547e-03 -0.0004972527 -0.0003044418  0.0007037652  0.002659380
2007-07-09  7.337565e-05  0.0022605055  0.0012172115 -0.0025157246 -0.001993871
2007-07-10  8.584371e-03  0.0057647029  0.0115503355 -0.0056582954  0.002805051
2007-07-11 -7.274844e-05  0.0027838359 -0.0029504763 -0.0026379885 -0.002952903
2007-07-12  3.485969e-03 -0.0009845428  0.0028903495  0.0008124302  0.004573291
2007-07-13 -7.252421e-04  0.0019435396  0.0000000000 -0.0020323145 -0.003391083
                  chfgbp        ghsusd        ghsgbp
2007-07-06  0.0001273483 -0.0033264938 -2.614079e-03
2007-07-09 -0.0009873737  0.0040086898  1.236683e-03
2007-07-10  0.0059364761 -0.0002705906 -5.801912e-03
2007-07-11 -0.0057330414  0.0026887042  4.994937e-05
2007-07-12  0.0037670817  0.0027212691  3.716576e-03
2007-07-13 -0.0020261005 -0.0051426368 -7.210559e-03

            eurusd  gbpusd  chfusd  usdgbp  eurgbp   chfgbp   ghsusd    ghsgbp
2007-07-05 1.35975 2.01155 0.82130 0.49715 0.67595 0.408303 1.074838 0.5343465
2007-07-06 1.36280 2.01055 0.82105 0.49750 0.67775 0.408355 1.071268 0.5329515
2007-07-09 1.36290 2.01510 0.82205 0.49625 0.67640 0.407952 1.075571 0.5336110
2007-07-10 1.37465 2.02675 0.83160 0.49345 0.67830 0.410381 1.075280 0.5305240
2007-07-11 1.37455 2.03240 0.82915 0.49215 0.67630 0.408035 1.078175 0.5305505
2007-07-12 1.37935 2.03040 0.83155 0.49255 0.67940 0.409575 1.081113 0.5325260

In [4]:
d <- na.omit(merge(cr_d,fr_d))
colnames(d) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")

In [5]:
d_p <- na.omit(merge(cp_d,fp_d))
colnames(d_p) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")

# Period 2: Data 2011/10 - 2015/11

In [6]:
d <- d["2011-10/2015-11"]
str(d)
head(d)
tail(d)

An xts object on 2011-10-03 / 2015-11-30 containing: 
  Data:    double [1038, 10]
  Columns: US Cocoa, London Cocoa, EURUSD, GBPUSD, CHFUSD ... with 5 more columns
  Index:   Date [1038] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:216] 37 42 100 101 122 123 124 128 142 162 ...
  ..- attr(*, "index")= num [1:216] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


               US Cocoa London Cocoa        EURUSD        GBPUSD       CHFUSD
2011-10-03 -0.013688427 -0.006386088 -0.0163058347 -0.0102872608 -0.014451912
2011-10-04  0.004202489  0.008698228  0.0128634905  0.0038499538  0.005695927
2011-10-05  0.011749235  0.006330956  0.0005995204 -0.0009045389 -0.007909542
2011-10-06  0.011985162  0.011409137  0.0063105827 -0.0020705951  0.002582074
2011-10-07 -0.002609508 -0.009689446 -0.0041029524  0.0078712623 -0.006606783
2011-10-10  0.001864629 -0.006320044  0.0196890265  0.0065017444  0.025220459
                 USDGBP       EURGBP       CHFGBP       GHSUSD       GHSGBP
2011-10-03  0.010386885 -0.005954831 -0.003981877 -0.004038530  0.006329860
2011-10-04 -0.004327673  0.008802925  0.001101991 -0.002488911 -0.006611036
2011-10-05  0.001393081  0.001565899 -0.006433088 -0.002783981 -0.001618611
2011-10-06  0.002008809  0.008425233  0.004402835  0.002164014  0.004302178
2011-10-07 -0.007981754 -0.012315498 -0.014256235 -0.001545939 -0.00953993

               US Cocoa London Cocoa        EURUSD       GBPUSD        CHFUSD
2015-11-20 -0.002668645  0.003061450 -0.0082320390 -0.006462959 -0.0058403965
2015-11-23 -0.018580097 -0.014515320 -0.0009397614 -0.004420115  0.0001527923
2015-11-24  0.009632826  0.013641576  0.0004699911 -0.002482500  0.0017300162
2015-11-25 -0.010237970 -0.011428696 -0.0013635831  0.002713885 -0.0045349241
2015-11-27  0.001814334  0.004409178 -0.0016506713 -0.004280104 -0.0069339501
2015-11-30  0.008723172  0.003513398 -0.0026940817  0.001262710  0.0020595209
                 USDGBP       EURGBP        CHFGBP       GHSUSD       GHSGBP
2015-11-20  0.006476953 -0.001568292  0.0006966214 -0.017947345 -0.011455044
2015-11-23  0.004395611  0.003418563  0.0045547614  0.007486908  0.011913969
2015-11-24  0.002416920  0.003194324  0.0041506593 -0.009952284 -0.007469305
2015-11-25 -0.002643806 -0.004830927 -0.0071591084  0.028716393  0.025993562
2015-11-27  0.004370439  0.002984653 -0.0024769732 -0.013235890 -0.00

# VAR-BEKK Models for the subsample

#### The estimation of the VAR-BEKK model is conducted in two stages, VAR model is the model specification for the mean equation (first stage) and BEKK is the specification for the variance equation (second stage).

## (i) Spillover between New York and London

In [7]:
cr_d <- cr_d["2011-10/2015-11"]
str(cr_d)
head(cr_d)

An xts object on 2011-10-03 / 2015-11-30 containing: 
  Data:    double [1038, 2]
  Columns: usc, lc
  Index:   Date [1038] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:159] 37 42 100 101 122 123 140 160 184 214 ...
  ..- attr(*, "index")= num [1:159] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                    usc           lc
2011-10-03 -0.013688427 -0.006386088
2011-10-04  0.004202489  0.008698228
2011-10-05  0.011749235  0.006330956
2011-10-06  0.011985162  0.011409137
2011-10-07 -0.002609508 -0.009689446
2011-10-10  0.001864629 -0.006320044

In [8]:
VARselect(cr_d)

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.906779e+01,-1.907550e+01,-1.907534e+01,-1.908523e+01,-1.908502e+01,-1.907875e+01,-1.908267e+01,-1.907815e+01,-1.907644e+01,-1.907729e+01
HQ(n),-1.905686e+01,-1.905728e+01,-1.904983e+01,-1.905243e+01,-1.904493e+01,-1.903137e+01,-1.902801e+01,-1.901619e+01,-1.900720e+01,-1.900076e+01
SC(n),-1.903899e+01,-1.902749e+01,-1.900813e+01,-1.899881e+01,-1.897940e+01,-1.895392e+01,-1.893865e+01,-1.891491e+01,-1.889401e+01,-1.887566e+01
FPE(n),5.235555e-09,5.195344e-09,5.196184e-09,5.145074e-09,5.146150e-09,5.178520e-09,5.158240e-09,5.181658e-09,5.190502e-09,5.186102e-09


In [9]:
var_cr<-VAR(cr_d,p = 4,type = "const")
summary(var_cr)
res_cr <- residuals(var_cr)
res_cr <- as.data.frame(res_cr)


VAR Estimation Results:
Endogenous variables: usc, lc 
Deterministic variables: const 
Sample size: 1034 
Log Likelihood: 6949.637 
Roots of the characteristic polynomial:
0.5842 0.5842 0.5832 0.5832 0.491 0.433 0.433 0.3079
Call:
VAR(y = cr_d, p = 4, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + lc.l1 + usc.l2 + lc.l2 + usc.l3 + lc.l3 + usc.l4 + lc.l4 + const 

         Estimate Std. Error t value Pr(>|t|)  
usc.l1  0.0387999  0.0762677   0.509   0.6110  
lc.l1   0.0196678  0.0861448   0.228   0.8195  
usc.l2 -0.0721081  0.0765240  -0.942   0.3463  
lc.l2   0.0620786  0.0862009   0.720   0.4716  
usc.l3  0.1358778  0.0761094   1.785   0.0745 .
lc.l3  -0.1658952  0.0857692  -1.934   0.0534 .
usc.l4 -0.0020958  0.0760928  -0.028   0.9780  
lc.l4  -0.1411454  0.0857832  -1.645   0.1002  
const   0.0001967  0.0004370   0.450   0.6527  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01401 on 1025 degrees of freedom


In [10]:
ind_var <- index(cr_d)[-c(1:4)]
str(ind_var)

 Date[1:1034], format: "2011-10-07" "2011-10-10" "2011-10-11" "2011-10-12" "2011-10-13" ...


In [11]:
res_cr <- xts(res_cr,order.by = ind_var)
str(res_cr)
head(res_cr)

An xts object on 2011-10-07 / 2015-11-30 containing: 
  Data:    double [1034, 2]
  Columns: usc, lc
  Index:   Date [1034] (TZ: "UTC")


                    usc           lc
2011-10-07 -0.003099505 -0.009961389
2011-10-10  0.002806055 -0.005969513
2011-10-11 -0.018113194 -0.013191439
2011-10-12  0.011940364 -0.001338381
2011-10-13 -0.004180043 -0.006704783
2011-10-14  0.015967754  0.010650408

In [12]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk_fit <- bekk_fit(spec = bekk_spec,data = res_cr)

In [13]:
bekk_fit$A
bekk_fit$A_t
bekk_fit$G
bekk_fit$G_t

0.17463723,0.08501612
0.01718164,0.12228524


3.9910162,2.080773
0.3454919,2.775230


0.94803092,-0.04378561
0.03630114,1.01897621


86.416001,-4.547753
2.641211,88.814276


### VAR-BEKK Model Table for US Cocoa - London Cocoa

In [14]:
bekkc <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_cr)$varresult$usc$coefficients["const","Estimate"],summary(var_cr)$varresult$lc$coefficients["const","Estimate"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Estimate"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l2","Estimate"],summary(var_cr)$varresult$usc$coefficients["lc.l2","Estimate"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l3","Estimate"],summary(var_cr)$varresult$usc$coefficients["lc.l3","Estimate"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l4","Estimate"],summary(var_cr)$varresult$usc$coefficients["lc.l4","Estimate"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Estimate"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l2","Estimate"],summary(var_cr)$varresult$lc$coefficients["lc.l2","Estimate"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l3","Estimate"],summary(var_cr)$varresult$lc$coefficients["lc.l3","Estimate"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l4","Estimate"],summary(var_cr)$varresult$lc$coefficients["lc.l4","Estimate"],
                                   bekk_fit$A[1,1],bekk_fit$A[1,2],bekk_fit$A[2,1],bekk_fit$A[2,2],bekk_fit$G[1,1],bekk_fit$G[1,2],bekk_fit$G[2,1],bekk_fit$G[2,2]),
                   Std_error = c(summary(var_cr)$varresult$usc$coefficients["const","Std. Error"],summary(var_cr)$varresult$lc$coefficients["const","Std. Error"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Std. Error"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l2","Std. Error"],summary(var_cr)$varresult$usc$coefficients["lc.l2","Std. Error"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l3","Std. Error"],summary(var_cr)$varresult$usc$coefficients["lc.l3","Std. Error"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l4","Std. Error"],summary(var_cr)$varresult$usc$coefficients["lc.l4","Std. Error"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Std. Error"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l2","Std. Error"],summary(var_cr)$varresult$lc$coefficients["lc.l2","Std. Error"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l3","Std. Error"],summary(var_cr)$varresult$lc$coefficients["lc.l3","Std. Error"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l4","Std. Error"],summary(var_cr)$varresult$lc$coefficients["lc.l4","Std. Error"],
                                   bekk_fit$A_sd[1,1],bekk_fit$A_sd[1,2],bekk_fit$A_sd[2,1],bekk_fit$A_sd[2,2],bekk_fit$G_sd[1,1],bekk_fit$G_sd[1,2],bekk_fit$G_sd[2,1],bekk_fit$G_sd[2,2]),
                   T_value = c(summary(var_cr)$varresult$usc$coefficients["const","t value"],summary(var_cr)$varresult$lc$coefficients["const","t value"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$usc$coefficients["lc.l1","t value"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l2","t value"],summary(var_cr)$varresult$usc$coefficients["lc.l2","t value"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l3","t value"],summary(var_cr)$varresult$usc$coefficients["lc.l3","t value"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l4","t value"],summary(var_cr)$varresult$usc$coefficients["lc.l4","t value"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$lc$coefficients["lc.l1","t value"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l2","t value"],summary(var_cr)$varresult$lc$coefficients["lc.l2","t value"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l3","t value"],summary(var_cr)$varresult$lc$coefficients["lc.l3","t value"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l4","t value"],summary(var_cr)$varresult$lc$coefficients["lc.l4","t value"],
                                   bekk_fit$A_t[1,1],bekk_fit$A_t[1,2],bekk_fit$A_t[2,1],bekk_fit$A_t[2,2],bekk_fit$G_t[1,1],bekk_fit$G_t[1,2],bekk_fit$G_t[2,1],bekk_fit$G_t[2,2]))

bekkc$Significant_at_5p_level[abs(bekkc$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.96 & abs(bekkc$T_value) < 2.58] <- "Stat significant at 5%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.645 & abs(bekkc$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) <= 1.645] <- "Not Statistically Significant"
bekkc

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,1.966947e-04,0.0004369523,0.4501514120,Not Statistically Significant
mu20,3.374724e-04,0.0003877170,0.8704089794,Not Statistically Significant
phi11,3.879991e-02,0.0762677210,0.5087330948,Not Statistically Significant
phi12,1.966779e-02,0.0861448442,0.2283106766,Not Statistically Significant
phi13,-7.210814e-02,0.0765240343,-0.9422939693,Not Statistically Significant
phi14,6.207856e-02,0.0862009482,0.7201610411,Not Statistically Significant
phi15,1.358778e-01,0.0761094187,1.7852948118,Stat significant at 10%
phi16,-1.658952e-01,0.0857691987,-1.9342043760,Stat significant at 10%
phi17,-2.095771e-03,0.0760927524,-0.0275423246,Not Statistically Significant


In [15]:
causality(var_cr,cause = "usc")$Granger
causality(var_cr,cause = "lc")$Granger


	Granger causality H0: usc do not Granger-cause lc

data:  VAR object var_cr
F-Test = 2.2149, df1 = 4, df2 = 2050, p-value = 0.0651



	Granger causality H0: lc do not Granger-cause usc

data:  VAR object var_cr
F-Test = 1.7589, df1 = 4, df2 = 2050, p-value = 0.1345


## (ii) VAR-BEKK Models for NY Cocoa

In [16]:
us_d <- na.omit(merge(cr_d[,1],fr_d))
str(us_d)

An xts object on 2011-10-03 / 2015-11-30 containing: 
  Data:    double [1038, 9]
  Columns: usc, eurusd, gbpusd, chfusd, usdgbp ... with 4 more columns
  Index:   Date [1038] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:3353] 1 2 3 4 5 6 7 8 9 10 ...
  ..- attr(*, "index")= num [1:3353] 1.18e+09 1.18e+09 1.18e+09 1.18e+09 1.18e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


## (a) US Cocoa - EURUSD

In [17]:
bekk1data <- us_d[,c(1:2)]
head(bekk1data)
tail(bekk1data)
VARselect(bekk1data,type = "const")

                    usc        eurusd
2011-10-03 -0.013688427 -0.0163058347
2011-10-04  0.004202489  0.0128634905
2011-10-05  0.011749235  0.0005995204
2011-10-06  0.011985162  0.0063105827
2011-10-07 -0.002609508 -0.0041029524
2011-10-10  0.001864629  0.0196890265

                    usc        eurusd
2015-11-20 -0.002668645 -0.0082320390
2015-11-23 -0.018580097 -0.0009397614
2015-11-24  0.009632826  0.0004699911
2015-11-25 -0.010237970 -0.0013635831
2015-11-27  0.001814334 -0.0016506713
2015-11-30  0.008723172 -0.0026940817

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.890708e+01,-1.889991e+01,-1.889369e+01,-1.890350e+01,-1.889956e+01,-1.889832e+01,-1.889408e+01,-1.888834e+01,-1.888289e+01,-1.887922e+01
HQ(n),-1.889614e+01,-1.888169e+01,-1.886818e+01,-1.887070e+01,-1.885947e+01,-1.885094e+01,-1.883942e+01,-1.882639e+01,-1.881364e+01,-1.880269e+01
SC(n),-1.887827e+01,-1.885190e+01,-1.882648e+01,-1.881708e+01,-1.879393e+01,-1.877349e+01,-1.875006e+01,-1.872511e+01,-1.870045e+01,-1.867758e+01
FPE(n),6.148390e-09,6.192613e-09,6.231228e-09,6.170445e-09,6.194811e-09,6.202480e-09,6.228809e-09,6.264694e-09,6.298979e-09,6.322120e-09


In [18]:
var_bekk1<-VAR(bekk1data,p = 1,type = "const")
res_bekk1 <- residuals(var_bekk1)
res_bekk1 <- as.data.frame(res_bekk1)

In [19]:
summary(var_bekk1)


VAR Estimation Results:
Endogenous variables: usc, eurusd 
Deterministic variables: const 
Sample size: 1037 
Log Likelihood: 6860.875 
Roots of the characteristic polynomial:
0.05546 0.03929
Call:
VAR(y = bekk1data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + eurusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)  
usc.l1     0.0568165  0.0315972   1.798   0.0724 .
eurusd.l1 -0.0111731  0.0785496  -0.142   0.8869  
const      0.0001624  0.0004390   0.370   0.7115  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01412 on 1034 degrees of freedom
Multiple R-Squared: 0.003155,	Adjusted R-squared: 0.001227 
F-statistic: 1.636 on 2 and 1034 DF,  p-value: 0.1952 


Estimation results for equation eurusd: 
eurusd = usc.l1 + eurusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1     0.0116717  0.0126728   0.921    0.357
eurusd.l1 -0.0406495  0.0315041  -1.290    0.197
const   

In [20]:
ind_var1 <- index(bekk1data)[-1]
str(ind_var1)

 Date[1:1037], format: "2011-10-04" "2011-10-05" "2011-10-06" "2011-10-07" "2011-10-10" ...


In [21]:
res_bekk1 <- xts(res_bekk1,order.by = ind_var1)

In [22]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk1_fit <- bekk_fit(spec = bekk_spec,data = res_bekk1)

In [23]:
bekk1_fit$A
bekk1_fit$A_t
bekk1_fit$G
bekk1_fit$G_t

0.17560725,-0.01096661
0.04247344,0.19635118


9.1689528,-1.003648
0.8529682,9.239717


0.981017489,0.002705376
-0.007741824,0.977067781


211.3085697,1.239057
-0.5863854,199.781122


### VAR-BEKK Model Table for US Cocoa - EURUSD

In [24]:
bekk1 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk1)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Estimate"],
                                   summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Estimate"],
                                   summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Estimate"],
                                   bekk1_fit$A[1,1],bekk1_fit$A[1,2],bekk1_fit$A[2,1],bekk1_fit$A[2,2],bekk1_fit$G[1,1],bekk1_fit$G[1,2],bekk1_fit$G[2,1],bekk1_fit$G[2,2]),
                   Std_error = c(summary(var_bekk1)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Std. Error"],
                                summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Std. Error"],
                                 summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Std. Error"],
                                 bekk1_fit$A_sd[1,1],bekk1_fit$A_sd[1,2],bekk1_fit$A_sd[2,1],bekk1_fit$A_sd[2,2],bekk1_fit$G_sd[1,1],bekk1_fit$G_sd[1,2],bekk1_fit$G_sd[2,1],bekk1_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk1)$varresult$usc$coefficients["const","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["const","t value"],
                       summary(var_bekk1)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","t value"],
                               summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","t value"],
                               bekk1_fit$A_t[1,1],bekk1_fit$A_t[1,2],bekk1_fit$A_t[2,1],bekk1_fit$A_t[2,2],bekk1_fit$G_t[1,1],bekk1_fit$G_t[1,2],bekk1_fit$G_t[2,1],bekk1_fit$G_t[2,2]))

bekk1$Significant_at_5p_level[abs(bekk1$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.96 & abs(bekk1$T_value) < 2.58] <- "Stat significant at 5%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.645 & abs(bekk1$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) <= 1.645] <- "Not Statistically Significant"
bekk1

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0001623897,0.0004389579,0.3699437,Not Statistically Significant
mu20,-0.0002380453,0.0001760540,-1.3521157,Not Statistically Significant
phi11,0.0568164766,0.0315971946,1.7981494,Stat significant at 10%
phi12,-0.0111730937,0.0785496388,-0.1422425,Not Statistically Significant
phi21,0.0116716508,0.0126727682,0.9210025,Not Statistically Significant
phi22,-0.0406494670,0.0315041059,-1.2902911,Not Statistically Significant
a11,0.1756072529,0.0191523784,9.1689528,Stat significant at 1%
a12,-0.0109666091,0.0109267514,-1.0036477,Not Statistically Significant
a21,0.0424734435,0.0497948705,0.8529682,Not Statistically Significant


In [25]:
causality(var_bekk1,cause = "usc")$Granger
causality(var_bekk1,cause = "eurusd")$Granger


	Granger causality H0: usc do not Granger-cause eurusd

data:  VAR object var_bekk1
F-Test = 0.84825, df1 = 1, df2 = 2068, p-value = 0.3572



	Granger causality H0: eurusd do not Granger-cause usc

data:  VAR object var_bekk1
F-Test = 0.020233, df1 = 1, df2 = 2068, p-value = 0.8869


## (b) VAR-BEKK: US Cocoa-GBPUSD

In [26]:
bekk2data <- us_d[,c(1,3)]
head(bekk2data)
tail(bekk2data)

                    usc        gbpusd
2011-10-03 -0.013688427 -0.0102872608
2011-10-04  0.004202489  0.0038499538
2011-10-05  0.011749235 -0.0009045389
2011-10-06  0.011985162 -0.0020705951
2011-10-07 -0.002609508  0.0078712623
2011-10-10  0.001864629  0.0065017444

                    usc       gbpusd
2015-11-20 -0.002668645 -0.006462959
2015-11-23 -0.018580097 -0.004420115
2015-11-24  0.009632826 -0.002482500
2015-11-25 -0.010237970  0.002713885
2015-11-27  0.001814334 -0.004280104
2015-11-30  0.008723172  0.001262710

In [27]:
VARselect(bekk2data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.936788e+01,-1.936286e+01,-1.935796e+01,-1.936902e+01,-1.936419e+01,-1.935931e+01,-1.935972e+01,-1.935684e+01,-1.935292e+01,-1.934704e+01
HQ(n),-1.935695e+01,-1.934463e+01,-1.933245e+01,-1.933622e+01,-1.932410e+01,-1.931193e+01,-1.930505e+01,-1.929488e+01,-1.928367e+01,-1.927051e+01
SC(n),-1.933908e+01,-1.931485e+01,-1.929074e+01,-1.928260e+01,-1.925857e+01,-1.923449e+01,-1.921569e+01,-1.919361e+01,-1.917048e+01,-1.914541e+01
FPE(n),3.878253e-09,3.897792e-09,3.916936e-09,3.873844e-09,3.892587e-09,3.911639e-09,3.910046e-09,3.921336e-09,3.936751e-09,3.959954e-09


In [28]:
var_bekk2<-VAR(bekk2data,p = 4,type = "const")
res_bekk2 <- residuals(var_bekk2)
res_bekk2 <- as.data.frame(res_bekk2)

In [29]:
summary(var_bekk2)


VAR Estimation Results:
Endogenous variables: usc, gbpusd 
Deterministic variables: const 
Sample size: 1034 
Log Likelihood: 7095.965 
Roots of the characteristic polynomial:
0.6166 0.6166 0.5895 0.5895 0.4041 0.4041 0.3726 0.3726
Call:
VAR(y = bekk2data, p = 4, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + gbpusd.l1 + usc.l2 + gbpusd.l2 + usc.l3 + gbpusd.l3 + usc.l4 + gbpusd.l4 + const 

            Estimate Std. Error t value Pr(>|t|)    
usc.l1     0.0504436  0.0317734   1.588    0.113    
gbpusd.l1  0.0486842  0.0995051   0.489    0.625    
usc.l2    -0.0330083  0.0317977  -1.038    0.299    
gbpusd.l2  0.1023854  0.0995311   1.029    0.304    
usc.l3    -0.0053477  0.0318191  -0.168    0.867    
gbpusd.l3  0.0834164  0.0994776   0.839    0.402    
usc.l4    -0.1339567  0.0318026  -4.212 2.75e-05 ***
gbpusd.l4  0.1330584  0.0992844   1.340    0.180    
const      0.0001716  0.0004367   0.393    0.694    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.0

In [30]:
ind_var2 <- index(bekk2data)[-c(1:4)]
str(ind_var2)

 Date[1:1034], format: "2011-10-07" "2011-10-10" "2011-10-11" "2011-10-12" "2011-10-13" ...


In [31]:
res_bekk2 <- xts(res_bekk2,order.by = ind_var2)

In [32]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk2_fit <- bekk_fit(spec = bekk_spec,data = res_bekk2)

In [33]:
bekk2_fit$A
bekk2_fit$A_t

0.17796530,0.002667489
0.07139057,0.167325921


8.3329607,0.3047656
0.8970658,7.2639282


In [34]:
bekk2_fit$G
bekk2_fit$G_t

0.980419392,-0.0004792078
-0.007178679,0.9803108872


199.5229768,-0.2400574
-0.3255008,175.3300042


### VAR-BEKK Model Table for US Cocoa - GBPUSD

In [35]:
bekk2 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk2)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Estimate"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Estimate"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l2","Estimate"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l2","Estimate"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l3","Estimate"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l3","Estimate"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l4","Estimate"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l4","Estimate"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Estimate"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l2","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l2","Estimate"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l3","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l3","Estimate"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l4","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l4","Estimate"],
                                   bekk2_fit$A[1,1],bekk2_fit$A[1,2],bekk2_fit$A[2,1],bekk2_fit$A[2,2],bekk2_fit$G[1,1],bekk2_fit$G[1,2],bekk2_fit$G[2,1],bekk2_fit$G[2,2]),
                   Std_error = c(summary(var_bekk2)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Std. Error"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Std. Error"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l2","Std. Error"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l2","Std. Error"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l3","Std. Error"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l3","Std. Error"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l4","Std. Error"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l4","Std. Error"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Std. Error"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l2","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l2","Std. Error"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l3","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l3","Std. Error"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l4","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l4","Std. Error"],
                                   bekk2_fit$A_sd[1,1],bekk2_fit$A_sd[1,2],bekk2_fit$A_sd[2,1],bekk2_fit$A_sd[2,2],bekk2_fit$G_sd[1,1],bekk2_fit$G_sd[1,2],bekk2_fit$G_sd[2,1],bekk2_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk2)$varresult$usc$coefficients["const","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","t value"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","t value"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l2","t value"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l2","t value"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l3","t value"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l3","t value"],
                                   summary(var_bekk2)$varresult$usc$coefficients["usc.l4","t value"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l4","t value"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","t value"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l2","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l2","t value"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l3","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l3","t value"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l4","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l4","t value"],
                                   bekk2_fit$A_t[1,1],bekk2_fit$A_t[1,2],bekk2_fit$A_t[2,1],bekk2_fit$A_t[2,2],bekk2_fit$G_t[1,1],bekk2_fit$G_t[1,2],bekk2_fit$G_t[2,1],bekk2_fit$G_t[2,2]))

bekk2$Significant_at_5p_level[abs(bekk2$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.96 & abs(bekk2$T_value) < 2.58] <- "Stat significant at 5%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.645 & abs(bekk2$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) <= 1.645] <- "Not Statistically Significant"
bekk2

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,1.715850e-04,0.0004367043,0.39290878,Not Statistically Significant
mu20,-1.855121e-05,0.0001406271,-0.13191778,Not Statistically Significant
phi11,5.044359e-02,0.0317733517,1.58760687,Not Statistically Significant
phi12,4.868423e-02,0.0995051011,0.48926368,Not Statistically Significant
phi13,-3.300827e-02,0.0317977282,-1.03807013,Not Statistically Significant
phi14,1.023854e-01,0.0995311328,1.02867696,Not Statistically Significant
phi15,-5.347705e-03,0.0318191187,-0.16806577,Not Statistically Significant
phi16,8.341641e-02,0.0994776203,0.83854451,Not Statistically Significant
phi17,-1.339567e-01,0.0318025778,-4.21213350,Stat significant at 1%


In [36]:
causality(var_bekk2,cause = "usc")$Granger
causality(var_bekk2,cause = "gbpusd")$Granger


	Granger causality H0: usc do not Granger-cause gbpusd

data:  VAR object var_bekk2
F-Test = 0.76941, df1 = 4, df2 = 2050, p-value = 0.5451



	Granger causality H0: gbpusd do not Granger-cause usc

data:  VAR object var_bekk2
F-Test = 0.90549, df1 = 4, df2 = 2050, p-value = 0.4598


## (c) VAR-BEKK: US Cocoa-CHFUSD

In [37]:
bekk3data <- us_d[,c(1,4)]
head(bekk3data)
tail(bekk3data)

                    usc       chfusd
2011-10-03 -0.013688427 -0.014451912
2011-10-04  0.004202489  0.005695927
2011-10-05  0.011749235 -0.007909542
2011-10-06  0.011985162  0.002582074
2011-10-07 -0.002609508 -0.006606783
2011-10-10  0.001864629  0.025220459

                    usc        chfusd
2015-11-20 -0.002668645 -0.0058403965
2015-11-23 -0.018580097  0.0001527923
2015-11-24  0.009632826  0.0017300162
2015-11-25 -0.010237970 -0.0045349241
2015-11-27  0.001814334 -0.0069339501
2015-11-30  0.008723172  0.0020595209

In [38]:
VARselect(bekk3data,type = "const",lag.max = 4)

,1,2,3,4
AIC(n),-1.820923e+01,-1.820239e+01,-1.819960e+01,-1.820688e+01
HQ(n),-1.819835e+01,-1.818426e+01,-1.817421e+01,-1.817424e+01
SC(n),-1.818056e+01,-1.815461e+01,-1.813270e+01,-1.812086e+01
FPE(n),1.235470e-08,1.243943e-08,1.247427e-08,1.238381e-08


In [39]:
var_bekk3<-VAR(bekk3data,p = 1,type = "const")
res_bekk3 <- residuals(var_bekk3)
res_bekk3 <- as.data.frame(res_bekk3)

In [40]:
ind_var3 <- index(bekk3data)[-1]
str(ind_var3)

 Date[1:1037], format: "2011-10-04" "2011-10-05" "2011-10-06" "2011-10-07" "2011-10-10" ...


In [41]:
res_bekk3 <- xts(res_bekk3,order.by = ind_var3)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk3_fit <- bekk_fit(spec = bekk_spec,data = res_bekk3)

In [42]:
bekk3_fit$A
bekk3_fit$A_t
bekk3_fit$G
bekk3_fit$G_t

0.19122012,-0.006424693
0.02615948,0.062678378


7.7235957,-0.1342102
0.6611992,1.2619372


0.978185476,0.008429576
0.002041117,0.890281454


86.9989447,0.5454431
0.0171078,7.0050058


### VAR-BEKK Model Table for US Cocoa - CHFUSD

In [43]:
bekk3 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk3)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Estimate"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Estimate"],
                                   summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Estimate"],
                                   bekk3_fit$A[1,1],bekk3_fit$A[1,2],bekk3_fit$A[2,1],bekk3_fit$A[2,2],bekk3_fit$G[1,1],bekk3_fit$G[1,2],bekk3_fit$G[2,1],bekk3_fit$G[2,2]),
                   Std_error = c(summary(var_bekk3)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Std. Error"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Std. Error"],
                                 summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Std. Error"],
                                 bekk3_fit$A_sd[1,1],bekk3_fit$A_sd[1,2],bekk3_fit$A_sd[2,1],bekk3_fit$A_sd[2,2],bekk3_fit$G_sd[1,1],bekk3_fit$G_sd[1,2],bekk3_fit$G_sd[2,1],bekk3_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk3)$varresult$usc$coefficients["const","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["const","t value"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","t value"],
                               summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","t value"],
                               bekk3_fit$A_t[1,1],bekk3_fit$A_t[1,2],bekk3_fit$A_t[2,1],bekk3_fit$A_t[2,2],bekk3_fit$G_t[1,1],bekk3_fit$G_t[1,2],bekk3_fit$G_t[2,1],bekk3_fit$G_t[2,2]))

bekk3$Significant_at_5p_level[abs(bekk3$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.96 & abs(bekk3$T_value) < 2.58] <- "Stat significant at 5%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.645 & abs(bekk3$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) <= 1.645] <- "Not Statistically Significant"
bekk3

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,1.628155e-04,0.0004385261,0.3712789,Not Statistically Significant
mu20,-8.504709e-05,0.0002448254,-0.3473785,Not Statistically Significant
phi11,5.732398e-02,0.0312226015,1.8359772,Stat significant at 10%
phi12,-2.225693e-02,0.0559296609,-0.3979450,Not Statistically Significant
phi21,1.058377e-02,0.0174313175,0.6071699,Not Statistically Significant
phi22,-1.688320e-02,0.0312250623,-0.5406938,Not Statistically Significant
a11,1.912201e-01,0.0247579146,7.7235957,Stat significant at 1%
a12,-6.424693e-03,0.0478703630,-0.1342102,Not Statistically Significant
a21,2.615948e-02,0.0395636922,0.6611992,Not Statistically Significant


In [44]:
causality(var_bekk3,cause = "usc")$Granger
causality(var_bekk3,cause = "chfusd")$Granger


	Granger causality H0: usc do not Granger-cause chfusd

data:  VAR object var_bekk3
F-Test = 0.36866, df1 = 1, df2 = 2068, p-value = 0.5438



	Granger causality H0: chfusd do not Granger-cause usc

data:  VAR object var_bekk3
F-Test = 0.15836, df1 = 1, df2 = 2068, p-value = 0.6907


## (d) VAR-BEKK model for US Cocoa - GHSUSD

In [45]:
bekk7data <- na.omit(merge(cr_d[,1],fr_d[,7]))
head(bekk7data)
tail(bekk7data)

                    usc       ghsusd
2011-10-03 -0.013688427 -0.004038530
2011-10-04  0.004202489 -0.002488911
2011-10-05  0.011749235 -0.002783981
2011-10-06  0.011985162  0.002164014
2011-10-07 -0.002609508 -0.001545939
2011-10-10  0.001864629 -0.006775137

                    usc       ghsusd
2015-11-20 -0.002668645 -0.017947345
2015-11-23 -0.018580097  0.007486908
2015-11-24  0.009632826 -0.009952284
2015-11-25 -0.010237970  0.028716393
2015-11-27  0.001814334 -0.013235890
2015-11-30  0.008723172  0.010970661

In [46]:
VARselect(bekk7data,type = "const",lag.max = 4)

,1,2,3,4
AIC(n),-1.750642e+01,-1.749959e+01,-1.750401e+01,-1.751157e+01
HQ(n),-1.749554e+01,-1.748146e+01,-1.747862e+01,-1.747893e+01
SC(n),-1.747775e+01,-1.745180e+01,-1.743711e+01,-1.742555e+01
FPE(n),2.494924e-08,2.512033e-08,2.500958e-08,2.482117e-08


In [47]:
var_bekk7<-VAR(bekk7data,p = 4,type = "const")
res_bekk7 <- residuals(var_bekk7)
res_bekk7 <- as.data.frame(res_bekk7)

In [48]:
summary(var_bekk7)


VAR Estimation Results:
Endogenous variables: usc, ghsusd 
Deterministic variables: const 
Sample size: 1034 
Log Likelihood: 6137.117 
Roots of the characteristic polynomial:
0.6004 0.6004 0.5828 0.5828 0.4839 0.4839 0.462 0.07612
Call:
VAR(y = bekk7data, p = 4, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + ghsusd.l1 + usc.l2 + ghsusd.l2 + usc.l3 + ghsusd.l3 + usc.l4 + ghsusd.l4 + const 

            Estimate Std. Error t value Pr(>|t|)    
usc.l1     0.0588163  0.0310203   1.896   0.0582 .  
ghsusd.l1  0.0616269  0.0394488   1.562   0.1185    
usc.l2    -0.0256425  0.0310629  -0.826   0.4093    
ghsusd.l2 -0.0077847  0.0399392  -0.195   0.8455    
usc.l3     0.0020055  0.0310684   0.065   0.9485    
ghsusd.l3  0.0129984  0.0400368   0.325   0.7455    
usc.l4    -0.1228635  0.0309866  -3.965 7.85e-05 ***
ghsusd.l4 -0.0119384  0.0396284  -0.301   0.7633    
const      0.0002061  0.0004427   0.466   0.6416    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.0

In [49]:
ind_var7 <- index(bekk7data)[-c(1:4)]
str(ind_var7)

 Date[1:1034], format: "2011-10-07" "2011-10-10" "2011-10-11" "2011-10-12" "2011-10-13" ...


In [50]:
res_bekk7 <- xts(res_bekk7,order.by = ind_var7)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk7_fit <- bekk_fit(spec = bekk_spec,data = res_bekk7)

In [51]:
bekk7_fit$A
bekk7_fit$A_t
bekk7_fit$G
bekk7_fit$G_t

0.19101642,0.03264131
-0.01862777,0.46305949


6.650071,3.183828
-0.546213,21.841993


0.962074146,-0.01410582
0.001370805,0.88668733


79.95625378,-2.145975
0.08816172,103.601112


### VAR-BEKK Model Table for US Cocoa - GHSUSD

In [52]:
bekk7 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk7)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l4","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l4","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l4","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l4","Estimate"],
                                   bekk7_fit$A[1,1],bekk7_fit$A[1,2],bekk7_fit$A[2,1],bekk7_fit$A[2,2],bekk7_fit$G[1,1],bekk7_fit$G[1,2],bekk7_fit$G[2,1],bekk7_fit$G[2,2]),
                   Std_error = c(summary(var_bekk7)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l4","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l4","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l4","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l4","Std. Error"],
                                   bekk7_fit$A_sd[1,1],bekk7_fit$A_sd[1,2],bekk7_fit$A_sd[2,1],bekk7_fit$A_sd[2,2],bekk7_fit$G_sd[1,1],bekk7_fit$G_sd[1,2],bekk7_fit$G_sd[2,1],bekk7_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk7)$varresult$usc$coefficients["const","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l4","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l4","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l4","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l4","t value"],
                                   bekk7_fit$A_t[1,1],bekk7_fit$A_t[1,2],bekk7_fit$A_t[2,1],bekk7_fit$A_t[2,2],bekk7_fit$G_t[1,1],bekk7_fit$G_t[1,2],bekk7_fit$G_t[2,1],bekk7_fit$G_t[2,2]))

bekk7$Significant_at_5p_level[abs(bekk7$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.96 & abs(bekk7$T_value) < 2.58] <- "Stat significant at 5%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.645 & abs(bekk7$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) <= 1.645] <- "Not Statistically Significant"
bekk7

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0002061023,0.0004427242,0.46553204,Not Statistically Significant
mu20,-0.0008560649,0.0003509303,-2.43941590,Stat significant at 5%
phi11,0.0588162648,0.0310202741,1.89605884,Stat significant at 10%
phi12,0.0616268975,0.0394488111,1.56219911,Not Statistically Significant
phi13,-0.0256424838,0.0310629342,-0.82550102,Not Statistically Significant
phi14,-0.0077847167,0.0399391958,-0.19491421,Not Statistically Significant
phi15,0.0020054571,0.0310684164,0.06454970,Not Statistically Significant
phi16,0.0129984123,0.0400367945,0.32466166,Not Statistically Significant
phi17,-0.1228634751,0.0309866068,-3.96505096,Stat significant at 1%


In [53]:
causality(var_bekk7,cause = "usc")$Granger
causality(var_bekk7,cause = "ghsusd")$Granger


	Granger causality H0: usc do not Granger-cause ghsusd

data:  VAR object var_bekk7
F-Test = 0.10326, df1 = 4, df2 = 2050, p-value = 0.9814



	Granger causality H0: ghsusd do not Granger-cause usc

data:  VAR object var_bekk7
F-Test = 0.7321, df1 = 4, df2 = 2050, p-value = 0.57


# (iii) VAR-BEKK Models for London Cocoa

In [54]:
l_d <- na.omit(merge(cr_d[,2],fr_d[,c(4:6)]))
str(l_d)
head(l_d)
tail(l_d)

An xts object on 2011-10-03 / 2015-11-30 containing: 
  Data:    double [1038, 4]
  Columns: lc, usdgbp, eurgbp, chfgbp
  Index:   Date [1038] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:3353] 1 2 3 4 5 6 7 8 9 10 ...
  ..- attr(*, "index")= num [1:3353] 1.18e+09 1.18e+09 1.18e+09 1.18e+09 1.18e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


                     lc       usdgbp       eurgbp       chfgbp
2011-10-03 -0.006386088  0.010386885 -0.005954831 -0.003981877
2011-10-04  0.008698228 -0.004327673  0.008802925  0.001101991
2011-10-05  0.006330956  0.001393081  0.001565899 -0.006433088
2011-10-06  0.011409137  0.002008809  0.008425233  0.004402835
2011-10-07 -0.009689446 -0.007981754 -0.012315498 -0.014256235
2011-10-10 -0.006320044 -0.006478577  0.013177096  0.018730388

                     lc       usdgbp       eurgbp        chfgbp
2015-11-20  0.003061450  0.006476953 -0.001568292  0.0006966214
2015-11-23 -0.014515320  0.004395611  0.003418563  0.0045547614
2015-11-24  0.013641576  0.002416920  0.003194324  0.0041506593
2015-11-25 -0.011428696 -0.002643806 -0.004830927 -0.0071591084
2015-11-27  0.004409178  0.004370439  0.002984653 -0.0024769732
2015-11-30  0.003513398 -0.001203732 -0.003482962  0.0007747134

## (a) VAR-BEKK: London-USDGBP

In [55]:
bekk4data <- l_d[,c(1,2)]
head(bekk4data)
tail(bekk4data)

                     lc       usdgbp
2011-10-03 -0.006386088  0.010386885
2011-10-04  0.008698228 -0.004327673
2011-10-05  0.006330956  0.001393081
2011-10-06  0.011409137  0.002008809
2011-10-07 -0.009689446 -0.007981754
2011-10-10 -0.006320044 -0.006478577

                     lc       usdgbp
2015-11-20  0.003061450  0.006476953
2015-11-23 -0.014515320  0.004395611
2015-11-24  0.013641576  0.002416920
2015-11-25 -0.011428696 -0.002643806
2015-11-27  0.004409178  0.004370439
2015-11-30  0.003513398 -0.001203732

In [56]:
VARselect(bekk4data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.955705e+01,-1.955317e+01,-1.955122e+01,-1.955564e+01,-1.955290e+01,-1.954856e+01,-1.954860e+01,-1.954328e+01,-1.953866e+01,-1.953584e+01
HQ(n),-1.954612e+01,-1.953494e+01,-1.952571e+01,-1.952284e+01,-1.951281e+01,-1.950118e+01,-1.949394e+01,-1.948132e+01,-1.946941e+01,-1.945931e+01
SC(n),-1.952824e+01,-1.950516e+01,-1.948401e+01,-1.946922e+01,-1.944728e+01,-1.942373e+01,-1.940458e+01,-1.938005e+01,-1.935622e+01,-1.933420e+01
FPE(n),3.209823e-09,3.222311e-09,3.228580e-09,3.214355e-09,3.223190e-09,3.237202e-09,3.237060e-09,3.254349e-09,3.269431e-09,3.278660e-09


In [57]:
var_bekk4<-VAR(bekk4data,p = 1,type = "const")
res_bekk4 <- residuals(var_bekk4)
res_bekk4 <- as.data.frame(res_bekk4)

In [58]:
summary(var_bekk4)


VAR Estimation Results:
Endogenous variables: lc, usdgbp 
Deterministic variables: const 
Sample size: 1037 
Log Likelihood: 7203.87 
Roots of the characteristic polynomial:
0.05663 0.02508
Call:
VAR(y = bekk4data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + usdgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)  
lc.l1     0.0605578  0.0310255   1.952   0.0512 .
usdgbp.l1 0.0805609  0.0862641   0.934   0.3506  
const     0.0002792  0.0003894   0.717   0.4735  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01253 on 1034 degrees of freedom
Multiple R-Squared: 0.004532,	Adjusted R-squared: 0.002606 
F-statistic: 2.354 on 2 and 1034 DF,  p-value: 0.09553 


Estimation results for equation usdgbp: 
usdgbp = lc.l1 + usdgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -4.173e-03  1.115e-02  -0.374    0.708
usdgbp.l1 -2.901e-02  3.101e-02  -0.936    0.350
const      1.415e

In [59]:
ind_var4 <- index(bekk4data)[-1]
str(ind_var4)

 Date[1:1037], format: "2011-10-04" "2011-10-05" "2011-10-06" "2011-10-07" "2011-10-10" ...


In [60]:
res_bekk4 <- xts(res_bekk4,order.by = ind_var4)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk4_fit <- bekk_fit(spec = bekk_spec,data = res_bekk4)

In [61]:
bekk4_fit$A
bekk4_fit$A_t
bekk4_fit$G
bekk4_fit$G_t

0.20079299,-0.002165827
-0.01633836,0.170094668


12.4218252,-0.2356423
-0.2576083,8.0632943


0.976326030,9.228426e-05
-0.001271311,9.799915e-01


253.43536269,0.04301071
-0.07265386,201.42945915


### VAR-BEKK Model Table for London Cocoa - USDGBP

In [62]:
bekk4 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk4)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Estimate"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Estimate"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Estimate"],
                                   bekk4_fit$A[1,1],bekk4_fit$A[1,2],bekk4_fit$A[2,1],bekk4_fit$A[2,2],bekk4_fit$G[1,1],bekk4_fit$G[1,2],bekk4_fit$G[2,1],bekk4_fit$G[2,2]),
                   Std_error = c(summary(var_bekk4)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Std. Error"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Std. Error"],
                                   bekk4_fit$A_sd[1,1],bekk4_fit$A_sd[1,2],bekk4_fit$A_sd[2,1],bekk4_fit$A_sd[2,2],bekk4_fit$G_sd[1,1],bekk4_fit$G_sd[1,2],bekk4_fit$G_sd[2,1],bekk4_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk4)$varresult$lc$coefficients["const","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","t value"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","t value"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","t value"],
                                   bekk4_fit$A_t[1,1],bekk4_fit$A_t[1,2],bekk4_fit$A_t[2,1],bekk4_fit$A_t[2,2],bekk4_fit$G_t[1,1],bekk4_fit$G_t[1,2],bekk4_fit$G_t[2,1],bekk4_fit$G_t[2,2]))

bekk4$Significant_at_5p_level[abs(bekk4$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.96 & abs(bekk4$T_value) < 2.58] <- "Stat significant at 5%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.645 & abs(bekk4$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) <= 1.645] <- "Not Statistically Significant"
bekk4

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,2.791595e-04,0.0003893501,0.71698837,Not Statistically Significant
mu20,1.414698e-05,0.0001399436,0.10109061,Not Statistically Significant
phi11,6.055778e-02,0.0310254511,1.95187422,Stat significant at 10%
phi12,8.056091e-02,0.0862640631,0.93388727,Not Statistically Significant
phi21,-4.172686e-03,0.0111514385,-0.37418364,Not Statistically Significant
phi22,-2.900926e-02,0.0310057826,-0.93560793,Not Statistically Significant
a11,2.007930e-01,0.0161645315,12.42182521,Stat significant at 1%
a12,-2.165827e-03,0.0091911641,-0.23564230,Not Statistically Significant
a21,-1.633836e-02,0.0634232673,-0.25760831,Not Statistically Significant


In [63]:
causality(var_bekk4,cause = "lc")$Granger
causality(var_bekk4,cause = "usdgbp")$Granger


	Granger causality H0: lc do not Granger-cause usdgbp

data:  VAR object var_bekk4
F-Test = 0.14001, df1 = 1, df2 = 2068, p-value = 0.7083



	Granger causality H0: usdgbp do not Granger-cause lc

data:  VAR object var_bekk4
F-Test = 0.87215, df1 = 1, df2 = 2068, p-value = 0.3505


# (b) VAR-BEKK: London Cocoa-EURGBP

In [64]:
bekk5data <- l_d[,c(1,3)]
head(bekk5data)

                     lc       eurgbp
2011-10-03 -0.006386088 -0.005954831
2011-10-04  0.008698228  0.008802925
2011-10-05  0.006330956  0.001565899
2011-10-06  0.011409137  0.008425233
2011-10-07 -0.009689446 -0.012315498
2011-10-10 -0.006320044  0.013177096

In [65]:
VARselect(bekk5data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.951219e+01,-1.950849e+01,-1.950714e+01,-1.951198e+01,-1.951121e+01,-1.950796e+01,-1.950621e+01,-1.950083e+01,-1.949606e+01,-1.949335e+01
HQ(n),-1.950126e+01,-1.949027e+01,-1.948163e+01,-1.947918e+01,-1.947112e+01,-1.946058e+01,-1.945155e+01,-1.943887e+01,-1.942681e+01,-1.941682e+01
SC(n),-1.948338e+01,-1.946048e+01,-1.943993e+01,-1.942556e+01,-1.940559e+01,-1.938313e+01,-1.936219e+01,-1.933759e+01,-1.931362e+01,-1.929171e+01
FPE(n),3.357095e-09,3.369541e-09,3.374086e-09,3.357807e-09,3.360380e-09,3.371344e-09,3.377232e-09,3.395483e-09,3.411712e-09,3.420973e-09


In [66]:
var_bekk5<-VAR(bekk5data,p = 1,type = "const")
res_bekk5 <- residuals(var_bekk5)
res_bekk5 <- as.data.frame(res_bekk5)

In [67]:
summary(var_bekk5)


VAR Estimation Results:
Endogenous variables: lc, eurgbp 
Deterministic variables: const 
Sample size: 1037 
Log Likelihood: 7174.914 
Roots of the characteristic polynomial:
0.06398 0.01277
Call:
VAR(y = bekk5data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + eurgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)  
lc.l1      0.0616629  0.0311012   1.983   0.0477 *
eurgbp.l1 -0.0375431  0.0841442  -0.446   0.6556  
const      0.0002728  0.0003899   0.700   0.4844  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01254 on 1034 degrees of freedom
Multiple R-Squared: 0.003884,	Adjusted R-squared: 0.001957 
F-statistic: 2.016 on 2 and 1034 DF,  p-value: 0.1337 


Estimation results for equation eurgbp: 
eurgbp = lc.l1 + eurgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -0.0045890  0.0115118  -0.399    0.690
eurgbp.l1 -0.0104559  0.0311451  -0.336    0.737
const     -0.

In [68]:
ind_var5 <- index(bekk5data)[-1]
str(ind_var5)

 Date[1:1037], format: "2011-10-04" "2011-10-05" "2011-10-06" "2011-10-07" "2011-10-10" ...


In [69]:
res_bekk5 <- xts(res_bekk5,order.by = ind_var5)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk5_fit <- bekk_fit(spec = bekk_spec,data = res_bekk5)

In [70]:
bekk5_fit$A
bekk5_fit$A_t
bekk5_fit$G
bekk5_fit$G_t

0.1920985,-0.0100027
-0.0261573,0.2175253


13.0015694,-1.210912
-0.4387909,9.606109


0.97854646,0.002217767
0.01236922,0.968611908


307.4284724,1.189389
0.7152718,143.626812


### VAR-BEKK Model Table for London-EURGBP

In [71]:
bekk5 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk5)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Estimate"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Estimate"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Estimate"],
                                   bekk5_fit$A[1,1],bekk5_fit$A[1,2],bekk5_fit$A[2,1],bekk5_fit$A[2,2],bekk5_fit$G[1,1],bekk5_fit$G[1,2],bekk5_fit$G[2,1],bekk5_fit$G[2,2]),
                   Std_error = c(summary(var_bekk5)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Std. Error"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Std. Error"],
                                   bekk5_fit$A_sd[1,1],bekk5_fit$A_sd[1,2],bekk5_fit$A_sd[2,1],bekk5_fit$A_sd[2,2],bekk5_fit$G_sd[1,1],bekk5_fit$G_sd[1,2],bekk5_fit$G_sd[2,1],bekk5_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk5)$varresult$lc$coefficients["const","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","t value"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","t value"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","t value"],
                                   bekk5_fit$A_t[1,1],bekk5_fit$A_t[1,2],bekk5_fit$A_t[2,1],bekk5_fit$A_t[2,2],bekk5_fit$G_t[1,1],bekk5_fit$G_t[1,2],bekk5_fit$G_t[2,1],bekk5_fit$G_t[2,2]))

bekk5$Significant_at_5p_level[abs(bekk5$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.96 & abs(bekk5$T_value) < 2.58] <- "Stat significant at 5%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.645 & abs(bekk5$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) <= 1.645] <- "Not Statistically Significant"
bekk5

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0002727597,0.0003899084,0.6995482,Not Statistically Significant
mu20,-0.0002107604,0.0001443204,-1.4603649,Not Statistically Significant
phi11,0.0616628715,0.0311011662,1.9826546,Stat significant at 5%
phi12,-0.0375431314,0.0841442074,-0.4461761,Not Statistically Significant
phi21,-0.0045889714,0.0115117581,-0.3986334,Not Statistically Significant
phi22,-0.0104558744,0.0311450624,-0.3357153,Not Statistically Significant
a11,0.1920984848,0.0147750228,13.0015694,Stat significant at 1%
a12,-0.0100027031,0.0082604680,-1.2109124,Not Statistically Significant
a21,-0.0261573036,0.0596122312,-0.4387909,Not Statistically Significant


In [72]:
causality(var_bekk5,cause = "lc")$Granger
causality(var_bekk5,cause = "eurgbp")$Granger


	Granger causality H0: lc do not Granger-cause eurgbp

data:  VAR object var_bekk5
F-Test = 0.15891, df1 = 1, df2 = 2068, p-value = 0.6902



	Granger causality H0: eurgbp do not Granger-cause lc

data:  VAR object var_bekk5
F-Test = 0.19907, df1 = 1, df2 = 2068, p-value = 0.6555


## (c) VAR-BEKK: London-CHFGBP

In [73]:
bekk6data <- l_d[,c(1,4)]
head(bekk6data)
tail(bekk6data)

                     lc       chfgbp
2011-10-03 -0.006386088 -0.003981877
2011-10-04  0.008698228  0.001101991
2011-10-05  0.006330956 -0.006433088
2011-10-06  0.011409137  0.004402835
2011-10-07 -0.009689446 -0.014256235
2011-10-10 -0.006320044  0.018730388

                     lc        chfgbp
2015-11-20  0.003061450  0.0006966214
2015-11-23 -0.014515320  0.0045547614
2015-11-24  0.013641576  0.0041506593
2015-11-25 -0.011428696 -0.0071591084
2015-11-27  0.004409178 -0.0024769732
2015-11-30  0.003513398  0.0007747134

In [74]:
VARselect(bekk6data,type = "const",lag.max = 4)

,1,2,3,4
AIC(n),-1.857683e+01,-1.857177e+01,-1.857069e+01,-1.857497e+01
HQ(n),-1.856595e+01,-1.855363e+01,-1.854530e+01,-1.854233e+01
SC(n),-1.854815e+01,-1.852398e+01,-1.850378e+01,-1.848895e+01
FPE(n),8.554352e-09,8.597732e-09,8.607036e-09,8.570253e-09


In [75]:
var_bekk6<-VAR(bekk6data,p = 1,type = "const")
res_bekk6 <- residuals(var_bekk6)
res_bekk6 <- as.data.frame(res_bekk6)

In [76]:
summary(var_bekk6)


VAR Estimation Results:
Endogenous variables: lc, chfgbp 
Deterministic variables: const 
Sample size: 1037 
Log Likelihood: 6696.958 
Roots of the characteristic polynomial:
0.06165 0.01178
Call:
VAR(y = bekk6data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + chfgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)  
lc.l1      0.0608727  0.0310442   1.961   0.0502 .
chfgbp.l1 -0.0094478  0.0531001  -0.178   0.8588  
const      0.0002802  0.0003895   0.719   0.4721  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01254 on 1034 degrees of freedom
Multiple R-Squared: 0.003723,	Adjusted R-squared: 0.001796 
F-statistic: 1.932 on 2 and 1034 DF,  p-value: 0.1454 


Estimation results for equation chfgbp: 
chfgbp = lc.l1 + chfgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -5.968e-03  1.818e-02  -0.328    0.743
chfgbp.l1 -1.101e-02  3.110e-02  -0.354    0.723
const     -7.

In [77]:
ind_var6 <- index(bekk6data)[-1]
str(ind_var6)

 Date[1:1037], format: "2011-10-04" "2011-10-05" "2011-10-06" "2011-10-07" "2011-10-10" ...


In [78]:
res_bekk6 <- xts(res_bekk6,order.by = ind_var6)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk6_fit <- bekk_fit(spec = bekk_spec,data = res_bekk6)

In [79]:
bekk6_fit$A
bekk6_fit$A_t
bekk6_fit$G
bekk6_fit$G_t

0.23824939,-0.01149037
-0.01242868,0.06340421


7.7431954,-0.2260017
-0.3554904,1.2534802


0.96500780,0.002437849
0.08224647,0.879923299


79.1370746,0.1389408
0.7090937,5.5732605


### VAR-BEKK Model Table for London-CHFGBP

In [80]:
bekk6 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk6)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Estimate"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Estimate"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Estimate"],
                                   bekk6_fit$A[1,1],bekk6_fit$A[1,2],bekk6_fit$A[2,1],bekk6_fit$A[2,2],bekk6_fit$G[1,1],bekk6_fit$G[1,2],bekk6_fit$G[2,1],bekk6_fit$G[2,2]),
                   Std_error = c(summary(var_bekk6)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Std. Error"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Std. Error"],
                                   bekk6_fit$A_sd[1,1],bekk6_fit$A_sd[1,2],bekk6_fit$A_sd[2,1],bekk6_fit$A_sd[2,2],bekk6_fit$G_sd[1,1],bekk6_fit$G_sd[1,2],bekk6_fit$G_sd[2,1],bekk6_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk6)$varresult$lc$coefficients["const","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","t value"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","t value"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","t value"],
                                   bekk6_fit$A_t[1,1],bekk6_fit$A_t[1,2],bekk6_fit$A_t[2,1],bekk6_fit$A_t[2,2],bekk6_fit$G_t[1,1],bekk6_fit$G_t[1,2],bekk6_fit$G_t[2,1],bekk6_fit$G_t[2,2]))

bekk6$Significant_at_5p_level[abs(bekk6$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.96 & abs(bekk6$T_value) < 2.58] <- "Stat significant at 5%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.645 & abs(bekk6$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) <= 1.645] <- "Not Statistically Significant"
bekk6

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0002802285,0.0003895269,0.7194074,Not Statistically Significant
mu20,-0.0000716071,0.0002281238,-0.3138958,Not Statistically Significant
phi11,0.0608727477,0.0310441525,1.9608442,Stat significant at 5%
phi12,-0.0094478373,0.0531001216,-0.1779250,Not Statistically Significant
phi21,-0.0059682110,0.0181807984,-0.3282700,Not Statistically Significant
phi22,-0.0110085053,0.0310977279,-0.3539971,Not Statistically Significant
a11,0.2382493879,0.0307688720,7.7431954,Stat significant at 1%
a12,-0.0114903671,0.0508419452,-0.2260017,Not Statistically Significant
a21,-0.0124286791,0.0349620680,-0.3554904,Not Statistically Significant


In [81]:
causality(var_bekk6,cause = "lc")$Granger
causality(var_bekk6,cause = "chfgbp")$Granger


	Granger causality H0: lc do not Granger-cause chfgbp

data:  VAR object var_bekk6
F-Test = 0.10776, df1 = 1, df2 = 2068, p-value = 0.7427



	Granger causality H0: chfgbp do not Granger-cause lc

data:  VAR object var_bekk6
F-Test = 0.031657, df1 = 1, df2 = 2068, p-value = 0.8588


## (d) VAR-BEKK Model for London Cocoa - GHSGBP

In [82]:
bekk8data <- na.omit(merge(cr_d[,2],fr_d[,8]))
head(bekk8data)
tail(bekk8data)

                     lc       ghsgbp
2011-10-03 -0.006386088  0.006329860
2011-10-04  0.008698228 -0.006611036
2011-10-05  0.006330956 -0.001618611
2011-10-06  0.011409137  0.004302178
2011-10-07 -0.009689446 -0.009539931
2011-10-10 -0.006320044 -0.013299207

                     lc       ghsgbp
2015-11-20  0.003061450 -0.011455044
2015-11-23 -0.014515320  0.011913969
2015-11-24  0.013641576 -0.007469305
2015-11-25 -0.011428696  0.025993562
2015-11-27  0.004409178 -0.008861253
2015-11-30  0.003513398  0.009705294

In [83]:
VARselect(bekk8data,type = "const",lag.max = 4)

,1,2,3,4
AIC(n),-1.752666e+01,-1.752156e+01,-1.753316e+01,-1.753737e+01
HQ(n),-1.751578e+01,-1.750343e+01,-1.750777e+01,-1.750473e+01
SC(n),-1.749799e+01,-1.747378e+01,-1.746625e+01,-1.745135e+01
FPE(n),2.444941e-08,2.457433e-08,2.429113e-08,2.418897e-08


In [84]:
var_bekk8<-VAR(bekk8data,p = 4,type = "const")
res_bekk8 <- residuals(var_bekk8)
res_bekk8 <- as.data.frame(res_bekk8)

In [85]:
summary(var_bekk8)


VAR Estimation Results:
Endogenous variables: lc, ghsgbp 
Deterministic variables: const 
Sample size: 1034 
Log Likelihood: 6150.455 
Roots of the characteristic polynomial:
0.6004 0.6004 0.5442 0.5442 0.5187 0.5187 0.5062 0.06012
Call:
VAR(y = bekk8data, p = 4, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + ghsgbp.l1 + lc.l2 + ghsgbp.l2 + lc.l3 + ghsgbp.l3 + lc.l4 + ghsgbp.l4 + const 

            Estimate Std. Error t value Pr(>|t|)    
lc.l1      0.0589115  0.0310446   1.898 0.058024 .  
ghsgbp.l1  0.0473016  0.0315292   1.500 0.133858    
lc.l2     -0.0538713  0.0310893  -1.733 0.083433 .  
ghsgbp.l2 -0.0036507  0.0314624  -0.116 0.907649    
lc.l3     -0.0285148  0.0310963  -0.917 0.359368    
ghsgbp.l3 -0.0006405  0.0315207  -0.020 0.983792    
lc.l4     -0.1072677  0.0310407  -3.456 0.000571 ***
ghsgbp.l4 -0.0184237  0.0316171  -0.583 0.560214    
const      0.0003304  0.0003917   0.843 0.399221    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 

In [86]:
ind_var8 <- index(bekk8data)[-c(1:4)]
str(ind_var8)

 Date[1:1034], format: "2011-10-07" "2011-10-10" "2011-10-11" "2011-10-12" "2011-10-13" ...


In [87]:
res_bekk8 <- xts(res_bekk8,order.by = ind_var8)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk8_fit <- bekk_fit(spec = bekk_spec,data = res_bekk8)

In [88]:
bekk8_fit$A
bekk8_fit$A_t
bekk8_fit$G
bekk8_fit$G_t

0.17973529,-0.009087744
-0.02104631,0.367401610


12.581586,-0.4741811
-1.508474,15.5363396


0.98193051,0.0001966072
0.01057014,0.9275021583


328.892758,0.03951398
2.572061,127.27829271


### VAR-BEKK Model Table for London Cocoa - GHSGBP

In [89]:
bekk8 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk8)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l4","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l4","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l4","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l4","Estimate"],
                                   bekk8_fit$A[1,1],bekk8_fit$A[1,2],bekk8_fit$A[2,1],bekk8_fit$A[2,2],bekk8_fit$G[1,1],bekk8_fit$G[1,2],bekk8_fit$G[2,1],bekk8_fit$G[2,2]),
                   Std_error = c(summary(var_bekk8)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l4","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l4","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l4","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l4","Std. Error"],
                                   bekk8_fit$A_sd[1,1],bekk8_fit$A_sd[1,2],bekk8_fit$A_sd[2,1],bekk8_fit$A_sd[2,2],bekk8_fit$G_sd[1,1],bekk8_fit$G_sd[1,2],bekk8_fit$G_sd[2,1],bekk8_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk8)$varresult$lc$coefficients["const","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l4","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l4","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l4","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l4","t value"],
                                   bekk8_fit$A_t[1,1],bekk8_fit$A_t[1,2],bekk8_fit$A_t[2,1],bekk8_fit$A_t[2,2],bekk8_fit$G_t[1,1],bekk8_fit$G_t[1,2],bekk8_fit$G_t[2,1],bekk8_fit$G_t[2,2]))

bekk8$Significant_at_5p_level[abs(bekk8$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.96 & abs(bekk8$T_value) < 2.58] <- "Stat significant at 5%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.645 & abs(bekk8$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) <= 1.645] <- "Not Statistically Significant"
bekk8

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0003303802,0.0003917407,0.84336462,Not Statistically Significant
mu20,-0.0007316616,0.0003882260,-1.88462829,Stat significant at 10%
phi11,0.0589114998,0.0310446044,1.89764054,Stat significant at 10%
phi12,0.0473015874,0.0315291557,1.50024910,Not Statistically Significant
phi13,-0.0538713024,0.0310893014,-1.73279231,Stat significant at 10%
phi14,-0.0036506784,0.0314623734,-0.11603315,Not Statistically Significant
phi15,-0.0285147761,0.0310963438,-0.91698163,Not Statistically Significant
phi16,-0.0006405135,0.0315207336,-0.02032039,Not Statistically Significant
phi17,-0.1072677197,0.0310407437,-3.45570714,Stat significant at 1%


In [90]:
causality(var_bekk8,cause = "lc")$Granger
causality(var_bekk8,cause = "ghsgbp")$Granger


	Granger causality H0: lc do not Granger-cause ghsgbp

data:  VAR object var_bekk8
F-Test = 0.14407, df1 = 4, df2 = 2050, p-value = 0.9656



	Granger causality H0: ghsgbp do not Granger-cause lc

data:  VAR object var_bekk8
F-Test = 0.62016, df1 = 4, df2 = 2050, p-value = 0.6482


# Summary of Period 2 models

In [91]:
bekk_c <- rep(NA,26)
for(i in 1:26){
    bekk_c[i] = paste0(round(bekkc[i,2],4)," (",round(bekkc[i,3],4),")")
}

bekk_1 <- rep(NA,14)
for(i in 1:14){
    bekk_1[i] = paste0(round(bekk1[i,2],4)," (",round(bekk1[i,3],4),")")
}

bekk_2 <- rep(NA,26)
for(i in 1:26){
    bekk_2[i] = paste0(round(bekk2[i,2],4)," (",round(bekk2[i,3],4),")")
}

bekk_3 <- rep(NA,14)
for(i in 1:14){
    bekk_3[i] = paste0(round(bekk3[i,2],4)," (",round(bekk3[i,3],4),")")
}

bekk_4 <- rep(NA,14)
for(i in 1:14){
    bekk_4[i] = paste0(round(bekk4[i,2],4)," (",round(bekk4[i,3],4),")")
}

bekk_5 <- rep(NA,14)
for(i in 1:14){
    bekk_5[i] = paste0(round(bekk5[i,2],4)," (",round(bekk5[i,3],4),")")
}

bekk_6 <- rep(NA,14)
for(i in 1:14){
    bekk_6[i] = paste0(round(bekk6[i,2],4)," (",round(bekk6[i,3],4),")")
}

bekk_7 <- rep(NA,26)
for(i in 1:26){
    bekk_7[i] = paste0(round(bekk7[i,2],4)," (",round(bekk7[i,3],4),")")
}


bekk_8 <- rep(NA,26)
for(i in 1:26){
    bekk_8[i] = paste0(round(bekk8[i,2],4)," (",round(bekk8[i,3],4),")")
}


In [92]:
parameters1 = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22")
parameters2 = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi17","phi18","phi21","phi22","phi23","phi24","phi25","phi26","phi27","phi28","a11","a12","a21","a22","g11","g12","g21","g22")

In [93]:
daily_bekk1 <- data.frame("Parameters" = parameters1,"BEKK1" = bekk_1,"BEKK3" = bekk_3,"BEKK4" = bekk_4,"BEKK5" = bekk_5,
                         "BEKK6" = bekk_6)
daily_bekk2 <- data.frame("Parameters" = parameters2,"BEKKC" = bekk_c,"BEKK2" = bekk_2,"BEKK7" = bekk_7,"BEKK8" = bekk_8)

In [94]:
daily_bekk1

Parameters,BEKK1,BEKK3,BEKK4,BEKK5,BEKK6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
mu10,2e-04 (4e-04),2e-04 (4e-04),3e-04 (4e-04),3e-04 (4e-04),3e-04 (4e-04)
mu20,-2e-04 (2e-04),-1e-04 (2e-04),0 (1e-04),-2e-04 (1e-04),-1e-04 (2e-04)
phi11,0.0568 (0.0316),0.0573 (0.0312),0.0606 (0.031),0.0617 (0.0311),0.0609 (0.031)
phi12,-0.0112 (0.0785),-0.0223 (0.0559),0.0806 (0.0863),-0.0375 (0.0841),-0.0094 (0.0531)
phi21,0.0117 (0.0127),0.0106 (0.0174),-0.0042 (0.0112),-0.0046 (0.0115),-0.006 (0.0182)
phi22,-0.0406 (0.0315),-0.0169 (0.0312),-0.029 (0.031),-0.0105 (0.0311),-0.011 (0.0311)
a11,0.1756 (0.0192),0.1912 (0.0248),0.2008 (0.0162),0.1921 (0.0148),0.2382 (0.0308)
a12,-0.011 (0.0109),-0.0064 (0.0479),-0.0022 (0.0092),-0.01 (0.0083),-0.0115 (0.0508)
a21,0.0425 (0.0498),0.0262 (0.0396),-0.0163 (0.0634),-0.0262 (0.0596),-0.0124 (0.035)


In [95]:
daily_bekk2

Parameters,BEKKC,BEKK2,BEKK7,BEKK8
<chr>,<chr>,<chr>,<chr>,<chr>
mu10,2e-04 (4e-04),2e-04 (4e-04),2e-04 (4e-04),3e-04 (4e-04)
mu20,3e-04 (4e-04),0 (1e-04),-9e-04 (4e-04),-7e-04 (4e-04)
phi11,0.0388 (0.0763),0.0504 (0.0318),0.0588 (0.031),0.0589 (0.031)
phi12,0.0197 (0.0861),0.0487 (0.0995),0.0616 (0.0394),0.0473 (0.0315)
phi13,-0.0721 (0.0765),-0.033 (0.0318),-0.0256 (0.0311),-0.0539 (0.0311)
phi14,0.0621 (0.0862),0.1024 (0.0995),-0.0078 (0.0399),-0.0037 (0.0315)
phi15,0.1359 (0.0761),-0.0053 (0.0318),0.002 (0.0311),-0.0285 (0.0311)
phi16,-0.1659 (0.0858),0.0834 (0.0995),0.013 (0.04),-6e-04 (0.0315)
phi17,-0.0021 (0.0761),-0.134 (0.0318),-0.1229 (0.031),-0.1073 (0.031)


# Final VAR-BEKK Table for Period 2 (Table A.3)

In [96]:
daily_bekk <- data.frame("Parameters" = parameters2,
                         "BEKKC"=bekk_c,
                         "BEKK1"=c(bekk_1[1:4],rep(NA,6),bekk_1[5:6],rep(NA,6),bekk_1[7:14]),
                         "BEKK2"=bekk_2,
                          "BEKK3"=c(bekk_3[1:4],rep(NA,6),bekk_3[5:6],rep(NA,6),bekk_3[7:14]),
                          "BEKK4"=c(bekk_4[1:4],rep(NA,6),bekk_4[5:6],rep(NA,6),bekk_4[7:14]),
                         "BEKK5"=c(bekk_5[1:4],rep(NA,6),bekk_5[5:6],rep(NA,6),bekk_5[7:14]),
                          "BEKK6"=c(bekk_6[1:4],rep(NA,6),bekk_6[5:6],rep(NA,6),bekk_6[7:14]),
                         "BEKK7" = bekk_7,
                         "BEKK8" = bekk_8)
daily_bekk

Parameters,BEKKC,BEKK1,BEKK2,BEKK3,BEKK4,BEKK5,BEKK6,BEKK7,BEKK8
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
mu10,2e-04 (4e-04),2e-04 (4e-04),2e-04 (4e-04),2e-04 (4e-04),3e-04 (4e-04),3e-04 (4e-04),3e-04 (4e-04),2e-04 (4e-04),3e-04 (4e-04)
mu20,3e-04 (4e-04),-2e-04 (2e-04),0 (1e-04),-1e-04 (2e-04),0 (1e-04),-2e-04 (1e-04),-1e-04 (2e-04),-9e-04 (4e-04),-7e-04 (4e-04)
phi11,0.0388 (0.0763),0.0568 (0.0316),0.0504 (0.0318),0.0573 (0.0312),0.0606 (0.031),0.0617 (0.0311),0.0609 (0.031),0.0588 (0.031),0.0589 (0.031)
phi12,0.0197 (0.0861),-0.0112 (0.0785),0.0487 (0.0995),-0.0223 (0.0559),0.0806 (0.0863),-0.0375 (0.0841),-0.0094 (0.0531),0.0616 (0.0394),0.0473 (0.0315)
phi13,-0.0721 (0.0765),NA,-0.033 (0.0318),NA,NA,NA,NA,-0.0256 (0.0311),-0.0539 (0.0311)
phi14,0.0621 (0.0862),NA,0.1024 (0.0995),NA,NA,NA,NA,-0.0078 (0.0399),-0.0037 (0.0315)
phi15,0.1359 (0.0761),NA,-0.0053 (0.0318),NA,NA,NA,NA,0.002 (0.0311),-0.0285 (0.0311)
phi16,-0.1659 (0.0858),NA,0.0834 (0.0995),NA,NA,NA,NA,0.013 (0.04),-6e-04 (0.0315)
phi17,-0.0021 (0.0761),NA,-0.134 (0.0318),NA,NA,NA,NA,-0.1229 (0.031),-0.1073 (0.031)
